In [1]:
import pandas as pd
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_test.tsv'


In [2]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Load train/val/test DataFrames
train_df = pd.read_csv(train_file, sep="\t")
dev_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")





In [3]:
l2id = {'None': 0, 'Society': 1, 'Organization': 2, 'Community': 3, 'Individual': 4}
id2l = {v: k for k, v in l2id.items()}


def clean_label(x):
    # handle missing or NaN → "None"
    if pd.isna(x) or x == 'None':
        return 'None'
    # already list-like e.g. ['Abusive']
    if isinstance(x, list):
        return x[0] if len(x) > 0 else 'None'
    # string cases like "[]" or "[Abusive]" or "[Political Hate]"
    x = x.strip("[]").strip()
    if x == "":
        return 'None'
    return x


def process_df(df):
    # Ensure labels are proper lists
    df["label"] = df["label"].apply(clean_label)
    df["label"] = df["label"].fillna("None")
    # Now create binary label
    df["toxic"] = df["label"].apply(lambda x: 0 if x == "None" else 1)
    df["label_id"] = df["label"].map(l2id)

    

  

    return df

train_df = process_df(train_df)
dev_df  = process_df(dev_df)

train_df

,id,text,label,toxic,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,0,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,0,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,1,4
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,0,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,1,4
...,...,...,...,...,...
35517,790325,তইওয়ানের এত ক্ষমতা হয়নি যে এক টুকরো জায়গা নষ্ট...,None,0,0
35518,328377,চুরের ঘরের চুর হালা,None,0,0
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Individual,1,4
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Organization,1,2


In [4]:

# Display the result
print(train_df[['text', 'toxic']].head())

                                                text  toxic
0  ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...      0
1  ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...      0
2          অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে      1
3  চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...      0
4  এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...      1


In [5]:
df_train = train_df

df_val = dev_df

df_val

,id,text,label,toxic,label_id
0,166449,ইন্ডিয়া কি মাছ ধরা বন্ধ রাখছেএক নদীতে দুইনীতি ...,Society,1,1
1,267692,লক্ষ টাকা ঘুষ দিয়ে অযোগ্য আর দায়িত্বহীন মানস...,Organization,1,2
2,184031,ওহা ভবনের দালাল,None,0,0
3,939131,আর কতো শিখবে আমার সোনার ছেলেরা এগুলো কে টাকা দ...,Individual,1,4
4,210284,কি সাংঘাতিক ভাই রে তুই,Individual,1,4
...,...,...,...,...,...
2507,350971,পুরোনো ইতিহাস তুলে ধরার জন্য সময় সংবাদ কে ধন্...,None,0,0
2508,539053,এই জন্যই আমাদের মেয়েরা কোরিয়া চলে যেতে চায়,None,0,0
2509,200314,এই শালা ইবলিশ এর বস এবলিশ এদের দেখে ভয় পায়,Individual,1,4
2510,788171,আমি কিনে ফেলছি আই ফোন ১৪,None,0,0


In [6]:
# import pandas as pd

# # existing train and dev
# df_train = train_df.copy()
# df_val   = dev_df.copy()

# # split dev_df in half
# half = len(df_val) // 2
# dev_train = df_val.iloc[:half].reset_index(drop=True)
# dev_val   = df_val.iloc[half:].reset_index(drop=True)

# # add first half of dev to training set
# df_train = pd.concat([df_train, dev_train], ignore_index=True)

# # final validation set is the second half
# df_val = dev_val

# print("Train size:", len(df_train))
# print("Validation size:", len(df_val))



import pandas as pd

# existing train and dev
df_train = train_df.copy()
df_val   = dev_df.copy()

# --- 75/25 split of dev_df ---
n   = len(df_val)
cut = int(0.5 * n)  # floor to an integer

# (optional) shuffle to avoid order bias:
# df_val = df_val.sample(frac=1, random_state=42).reset_index(drop=True)

dev_train = df_val.iloc[:cut].reset_index(drop=True)  # 75%
dev_val   = df_val.iloc[cut:].reset_index(drop=True)  # 25%

# add 75% of dev to training set
df_train = pd.concat([df_train, dev_train], ignore_index=True)

# final validation set is the remaining 25%
df_val = dev_val

print("Train size:", len(df_train))
print("Validation size:", len(df_val))


Train size: 36778
Validation size: 1256


In [7]:
toxic_df = df_train
target_list = sorted(toxic_df['label'].unique().tolist()) # Sort for consistent column order
print(f"Target Categories: {target_list}")

Target Categories: ['Community', 'Individual', 'None', 'Organization', 'Society']


In [8]:
df_train = pd.get_dummies(df_train, columns=['label'], prefix='', prefix_sep='')[['text'] + target_list]

df_val = pd.get_dummies(df_val, columns=['label'], prefix='', prefix_sep='')[['text'] + target_list]

df_val

,text,Community,Individual,None,Organization,Society
0,মিথ্যা নাটক করে লাভ নেই সাজানো নাটক শুরু হয়ে গেছে,False,True,False,False,False
1,ঘন্টা বাজাতে দফতরের কাজ টা কি আপনি নেবেন নাকি,False,False,True,False,False
2,ইতিহাসের দীর্ঘতম বিসিএস শেষে মেধাবীদের সাথে এভ...,False,False,False,True,False
3,সময় টিভি আসলে কি চায় ভারত সিদ্ধান্ত মেনে নিলে ...,False,False,False,True,False
4,আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি,False,False,False,True,False
...,...,...,...,...,...,...
1251,পুরোনো ইতিহাস তুলে ধরার জন্য সময় সংবাদ কে ধন্...,False,False,True,False,False
1252,এই জন্যই আমাদের মেয়েরা কোরিয়া চলে যেতে চায়,False,False,True,False,False
1253,এই শালা ইবলিশ এর বস এবলিশ এদের দেখে ভয় পায়,False,True,False,False,False
1254,আমি কিনে ফেলছি আই ফোন ১৪,False,False,True,False,False


In [9]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel
from collections import defaultdict
from tqdm import tqdm as tq
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoConfig, AutoModel
from torch.optim import AdamW  # <-- use this instead


MODEL_NAME ="csebuetnlp/banglabert_large"
# Use a pipeline as a high-level helper
#MODEL_NAME = "google-bert/bert-base-multilingual-cased"

#MODEL_NAME="FacebookAI/xlm-roberta-large"

# You can keep these as they are or tune them
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 2e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from collections import defaultdict
from tqdm import tqdm as tq
from sklearn.model_selection import train_test_split

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        # Corrected column name from 'title' or 'Text' to 'text'
        self.texts = list(df['text']) 
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'text': text
        }

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
val_dataset = CustomDataset(df_val, tokenizer, MAX_LEN, target_list)

train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm as tq

# # Your existing configuration
# MODEL_NAME = "csebuetnlp/banglabert"
# MAX_LEN = 256
# TRAIN_BATCH_SIZE = 16
# VAL_BATCH_SIZE = 16
# EPOCHS = 7
# LEARNING_RATE = 2e-5
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TACT specific hyperparameters
EPSILON = 0.01  # Perturbation factor
LAMBDA = 0.3    # Balance parameter between BCE and InfoNCE
TEMPERATURE = 0.07  # Temperature for InfoNCE loss

class TACTBanglaBERT(nn.Module):
    def __init__(self, model_name, num_classes, hidden_dim=768):
        super(TACTBanglaBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        
        # Multi-label classification head
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
        # Non-linear projection layer for contrastive learning
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 128)  # Final projection dimension
        )
        
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        
    def forward(self, input_ids, attention_mask, token_type_ids=None, targets=None, apply_tact=True):
        # Get BERT embeddings
        outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True
        )
        
        # Use [CLS] token representation from last hidden state
        # BanglaBERT doesn't have pooler_output, so we use the [CLS] token (first token)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, 0, :]  # [CLS] token is at index 0
        
        if apply_tact and targets is not None and self.training:
            return self.forward_with_tact(pooled_output, targets)
        else:
            # Standard forward pass
            pooled_output = self.dropout(pooled_output)
            logits = self.classifier(pooled_output)
            
            loss = None
            if targets is not None:
                # Multi-label binary cross entropy
                loss = F.binary_cross_entropy_with_logits(logits, targets)
                
            return {
                "loss": loss, 
                "logits": logits, 
                "hidden_states": pooled_output
            }
    
    def forward_with_tact(self, original_repr, targets):
        """
        Implements TACT training with token-level adversarial perturbations
        """
        # Enable gradients for token representations
        original_repr = original_repr.requires_grad_(True)
        
        # Initial forward pass to compute gradients
        pooled_output = self.dropout(original_repr)
        logits = self.classifier(pooled_output)
        
        # Compute binary cross-entropy loss
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets)
        
        # Compute gradients w.r.t. token representations
        grad_outputs = torch.ones_like(bce_loss)
        grad = torch.autograd.grad(
            outputs=bce_loss,
            inputs=original_repr,
            grad_outputs=grad_outputs,
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]
        
        # Calculate adversarial perturbation (Equation 5 from TACT paper)
        grad_norm = torch.norm(grad, dim=-1, keepdim=True)
        grad_norm = torch.clamp(grad_norm, min=1e-8)  # Avoid division by zero
        perturbation = -EPSILON * (grad / grad_norm)
        
        # Generate perturbed representation (Equation 6)
        perturbed_repr = original_repr + perturbation
        
        # Forward pass with original representation
        pooled_original = self.dropout(original_repr)
        logits_original = self.classifier(pooled_original)
        proj_original = self.projection(pooled_original)
        
        # Forward pass with perturbed representation  
        pooled_perturbed = self.dropout(perturbed_repr)
        logits_perturbed = self.classifier(pooled_perturbed)
        proj_perturbed = self.projection(pooled_perturbed)
        
        # Compute losses
        bce_loss_original = F.binary_cross_entropy_with_logits(logits_original, targets)
        bce_loss_perturbed = F.binary_cross_entropy_with_logits(logits_perturbed, targets)
        
        # InfoNCE Loss for contrastive learning
        infonce_loss = self.compute_infonce_loss(proj_original, proj_perturbed)
        
        # Combined loss (Equation 7 from TACT paper)
        total_loss = (1 - LAMBDA) / 2 * (bce_loss_original + bce_loss_perturbed) + LAMBDA * infonce_loss
        
        return {
            "loss": total_loss,
            "logits": logits_original,
            "hidden_states": pooled_original,
            "bce_loss_original": bce_loss_original,
            "bce_loss_perturbed": bce_loss_perturbed,
            "infonce_loss": infonce_loss
        }
    
    def compute_infonce_loss(self, z1, z2):
        """
        Compute InfoNCE loss for contrastive learning
        """
        batch_size = z1.size(0)
        
        # Normalize embeddings
        z1_norm = F.normalize(z1, dim=-1)
        z2_norm = F.normalize(z2, dim=-1)
        
        # Concatenate representations
        representations = torch.cat([z1_norm, z2_norm], dim=0)
        
        # Compute similarity matrix
        similarity_matrix = torch.matmul(representations, representations.T) / TEMPERATURE
        
        # Create positive pair labels
        labels = torch.cat([torch.arange(batch_size, 2*batch_size), 
                           torch.arange(0, batch_size)]).to(device)
        
        # Mask diagonal (self-similarity)
        mask = torch.eye(2*batch_size, dtype=torch.bool, device=device)
        similarity_matrix.masked_fill_(mask, -float('inf'))
        
        # InfoNCE loss
        loss = F.cross_entropy(similarity_matrix, labels)
        
        return loss

class TACTTrainer:
    def __init__(self, model, train_loader, val_loader, optimizer, scheduler=None):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_accuracy = 0.0
        
    def train_epoch(self):
        self.model.train()
        
        total_loss = 0
        total_bce_original = 0
        total_bce_perturbed = 0
        total_infonce = 0
        num_batches = 0
        
        progress_bar = tq(self.train_loader, desc="Training")
        
        for batch in progress_bar:
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            targets = batch['targets'].to(device)
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass with TACT
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                targets=targets,
                apply_tact=True
            )
            
            loss = outputs['loss']
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            if self.scheduler:
                self.scheduler.step()
            
            # Update metrics
            total_loss += loss.item()
            if 'bce_loss_original' in outputs:
                total_bce_original += outputs['bce_loss_original'].item()
            if 'bce_loss_perturbed' in outputs:
                total_bce_perturbed += outputs['bce_loss_perturbed'].item()
            if 'infonce_loss' in outputs:
                total_infonce += outputs['infonce_loss'].item()
            
            num_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        # Calculate averages
        metrics = {
            'avg_loss': total_loss / num_batches,
            'avg_bce_original': total_bce_original / num_batches,
            'avg_bce_perturbed': total_bce_perturbed / num_batches,
            'avg_infonce': total_infonce / num_batches
        }
        
        return metrics
    
    def evaluate(self):
        self.model.eval()
        
        total_loss = 0
        all_predictions = []
        all_targets = []
        num_batches = 0
        
        progress_bar = tq(self.val_loader, desc="Validation")
        
        with torch.no_grad():
            for batch in progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                targets = batch['targets'].to(device)
                
                # Forward pass without TACT
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    targets=targets,
                    apply_tact=False
                )
                
                loss = outputs['loss']
                total_loss += loss.item()
                num_batches += 1
                
                # Get predictions
                logits = outputs['logits']
                predictions = torch.sigmoid(logits)
                predicted_labels = (predictions > 0.5).float()
                
                all_predictions.extend(predicted_labels.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
                progress_bar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
        
        # Calculate metrics
        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        # Exact match accuracy (all labels must match)
        exact_match_accuracy = accuracy_score(all_targets, all_predictions)
        
        # Per-label accuracy
        label_accuracies = []
        for i in range(all_targets.shape[1]):
            label_acc = accuracy_score(all_targets[:, i], all_predictions[:, i])
            label_accuracies.append(label_acc)
        
        avg_loss = total_loss / num_batches
        
        return avg_loss, exact_match_accuracy, label_accuracies

def train_tact_model(train_data_loader, val_data_loader, target_list, num_epochs=EPOCHS):
    """
    Train TACT model using your existing data loaders
    """
    print(f"Training TACT model with {len(target_list)} labels: {target_list}")
    print(f"Device: {device}")
    
    # Initialize model
    model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # Optimizer with weight decay
    optimizer = AdamW(
        model.parameters(), 
        lr=LEARNING_RATE, 
        weight_decay=0.01,
        eps=1e-8
    )
    
    # Learning rate scheduler
    total_steps = len(train_data_loader) * num_epochs
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=1.0, 
        end_factor=0.1, 
        total_iters=total_steps
    )
    
    # Initialize trainer
    trainer = TACTTrainer(model, train_data_loader, val_data_loader, optimizer, scheduler)
    
    print(f"\nStarting training for {num_epochs} epochs...")
    print("=" * 80)
    
    # Training loop
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 60)
        
        # Train
        train_metrics = trainer.train_epoch()
        print(f"Training - Loss: {train_metrics['avg_loss']:.4f}, "
              f"BCE Orig: {train_metrics['avg_bce_original']:.4f}, "
              f"BCE Pert: {train_metrics['avg_bce_perturbed']:.4f}, "
              f"InfoNCE: {train_metrics['avg_infonce']:.4f}")
        
        # Evaluate
        val_loss, exact_accuracy, label_accuracies = trainer.evaluate()
        print(f"Validation - Loss: {val_loss:.4f}, Exact Match Accuracy: {exact_accuracy:.4f}")
        
        # Print per-label accuracies
        print("Per-label accuracies:")
        for i, (label_name, acc) in enumerate(zip(target_list, label_accuracies)):
            print(f"  {label_name}: {acc:.4f}")
        
        # Save best model
        if exact_accuracy > best_accuracy:
            best_accuracy = exact_accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_accuracy': best_accuracy,
                'target_list': target_list
            }, 'best_tact_bangla_model.pth')
            print(f"✓ New best model saved! Accuracy: {best_accuracy:.4f}")
    
    print(f"\nTraining completed! Best exact match accuracy: {best_accuracy:.4f}")
    return model, trainer

# Inference function for your trained model
def predict_with_tact(model, text, tokenizer, target_list, max_len=MAX_LEN, threshold=0.5):
    """
    Make predictions using the trained TACT model
    """
    model.eval()
    
    # Tokenize text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            apply_tact=False
        )
        
        # Get probabilities
        logits = outputs['logits']
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        
        # Create result dictionary
        results = {}
        for i, (label, prob) in enumerate(zip(target_list, probabilities)):
            results[label] = {
                'probability': float(prob),
                'predicted': bool(prob > threshold)
            }
    
    return results

# Usage with your existing setup:
"""
# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model, trainer = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
print(predictions)

# Load saved model for inference
checkpoint = torch.load('best_tact_bangla_model.pth')
model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
"""

# Usage with your existing setup:

# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model, trainer = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
# sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
# predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
# print(predictions)

# # Load saved model for inference
# checkpoint = torch.load('best_tact_bangla_model.pth')
# model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
# model.load_state_dict(checkpoint['model_state_dict'])

In [36]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer



# load checkpoint + target list
checkpoint = torch.load('best_tact_bangla_model.pth', map_location=device)
target_list = checkpoint['target_list']
num_labels = len(target_list)

In [55]:
# inference dataset
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {k: v.squeeze(0) for k, v in enc.items()}

# dataloader
test_ds = TestDataset(test_df['text'].tolist(), tokenizer, MAX_LEN)
test_loader = DataLoader(test_ds, batch_size=VAL_BATCH_SIZE, shuffle=False)

In [56]:

print("\n--- Evaluating Pipeline on Test Set ---")

model = TACTBanglaBERT(MODEL_NAME, num_labels).to(device)
model.load_state_dict(checkpoint['model_state_dict'])



model.eval()




--- Evaluating Pipeline on Test Set ---


TACTBanglaBERT(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [57]:

probs_all = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch.get('token_type_ids')
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids, device=device)
        else:
            token_type_ids = token_type_ids.to(device)

        out = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            apply_tact=False
        )
        logits = out['logits']
        probs = torch.sigmoid(logits).cpu().numpy()  # shape [B, num_labels]
        probs_all.append(probs)

probs_all = np.vstack(probs_all)  # shape [N, num_labels]

# MAX-PROB (Top-1) labeling
top1_idx   = probs_all.argmax(axis=1)
top1_label = [target_list[i] for i in top1_idx]
top1_prob  = probs_all.max(axis=1)

# ---------------------------------------------------------------------
# Assemble output and save
# ---------------------------------------------------------------------
prob_cols = [f'prob_{l}' for l in target_list]
pred_df = pd.DataFrame(probs_all, columns=prob_cols)
pred_df['pred_label'] = top1_label
pred_df['pred_prob']  = top1_prob

In [58]:
pred_df

,prob_Community,prob_Individual,prob_None,prob_Organization,prob_Society,pred_label,pred_prob
0,0.077769,0.015923,0.097183,0.035300,0.818726,Society,0.818726
1,0.005938,0.063930,0.945504,0.002170,0.002271,None,0.945504
2,0.266807,0.010014,0.669219,0.027643,0.021974,None,0.669219
3,0.771336,0.049234,0.198474,0.017351,0.029835,Community,0.771336
4,0.312005,0.614860,0.091083,0.004351,0.030483,Individual,0.614860
...,...,...,...,...,...,...,...
10195,0.228129,0.161033,0.031235,0.022525,0.606961,Society,0.606961
10196,0.012582,0.967270,0.027649,0.011332,0.011724,Individual,0.967270
10197,0.155134,0.856026,0.035680,0.010399,0.056156,Individual,0.856026
10198,0.022920,0.012747,0.907622,0.051957,0.034911,None,0.907622


In [59]:
# import numpy as np
# import pandas as pd


# #'Abusive', 'None', 'Political Hate', 'Profane', 'Religious Hate', 'Sexism'
# # #'Abusive', 'Political Hate', 'Profane', 'Religious Hate', 'Sexism']
# # 'Community', 'Individual', 'None', 'Organization', 'Society'
# # ['Community', 'Individual', 'None', 'Organization', 'Society']
# # Your mapping
# id2l = {
#     0: 'Community',
#     1: 'Individual',
#     2: 'None',
#     3: 'Organization',
#     4: 'Society'
   
# }

# # Example y_pred
# # y_pred = np.array([[0,0,0,0,0],[0,0,0,1,0],[1,0,0,0,0]])

# def decode_labels(row):
#     indices = np.where(row == 1)[0]
#     if len(indices) == 0:
#         return "None"
#     # If multiple labels, join them with comma
#     return ", ".join([id2l[i] for i in indices])

# # Convert predictions into a DataFrame column
# df = pd.DataFrame()
# df["Predicted_Label"] = [decode_labels(row) for row in y_pred]

# print(df.head())


In [60]:
test_df['label']=pred_df['pred_label']
test_df['model']='TACTBanglaBERT'
test_df['label'].unique()

array(['Society', 'None', 'Community', 'Individual', 'Organization'],
      dtype=object)

In [61]:
test_df = test_df[['id', 'label', 'model']]
test_df

,id,label,model
0,12764,Society,TACTBanglaBERT
1,202933,None,TACTBanglaBERT
2,165894,None,TACTBanglaBERT
3,124999,Community,TACTBanglaBERT
4,535301,Individual,TACTBanglaBERT
...,...,...,...
10195,908819,Society,TACTBanglaBERT
10196,597085,Individual,TACTBanglaBERT
10197,901448,Individual,TACTBanglaBERT
10198,617821,None,TACTBanglaBERT


In [62]:
test_df.to_csv("V6_subtask_2A.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")

Saved to final_ensemble.tsv
